In [1]:
import argparse
import os
import sys
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from tqdm import tqdm
from torchsummary import summary
import tabulate
import torchvision.models as models
import cv2
import numpy as np
import torch.utils.data as utils

In [2]:
data_path = './data/resnext/40X/'

In [3]:
x = []
y = []
for l in os.listdir(data_path):
    tempx = np.load(data_path+l)
    tempy = (l[4]=='M')
    x.append(tempx)
    y.append(tempy)

In [4]:
data = list(zip(x, y))
data = np.array(data)
len(data)

1995

In [5]:
count = 0
for i in range(1995):
    if(np.array_equal(x[i], data[i][0]) and y[i]==(data[i][1])):
        count += 1
count

1995

In [6]:
# model = torch.nn.Sequential()
# model = torch.nn.Linear(2048, 1024)
# model = F.relu(model)
# model = torch.nn.Linear(1024, 512)
# model = F.relu(model)
# model = torch.nn.Linear(512, 64)
# model = F.relu(model)
# model = torch.nn.Linear(64, 2)
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 64)
        self.fc4 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        x = torch.sigmoid(x)
        return x

In [7]:
model = Model()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=0.001,
    weight_decay=5e-4
)

In [8]:
np.random.shuffle(data)

In [9]:
data.shape

(1995, 2)

In [10]:
train = data[:1500]
train_x, train_y = [], []
for i in train:
    train_x.append(np.squeeze(i[0]))
    train_y.append([float(i[1])])
train_x = np.array(train_x)
train_y = np.array(train_y)
val = data[1500:1800]
val_x, val_y = [], []
for i in val:
    val_x.append(np.squeeze(i[0]))
    val_y.append([float(i[1])])
val_x = np.array(val_x)
val_y = np.array(val_y)
test = data[1800:]
test_x, test_y = [], []
for i in test:
    test_x.append(np.squeeze(i[0]))
    test_y.append([float(i[1])])
test_x = np.array(test_x)
test_y = np.array(test_y)

In [11]:
train_x.shape, train_y.shape, val_x.shape, val_y.shape, test_x.shape, test_y.shape

((1500, 2048), (1500, 1), (300, 2048), (300, 1), (195, 2048), (195, 1))

In [12]:
t_x = torch.stack([torch.Tensor(i) for i in train_x]) # transform to torch tensors
t_y = torch.stack([torch.Tensor(i) for i in train_y])
my_dataset = utils.TensorDataset(t_x,t_y) # create your datset
train_dataloader = utils.DataLoader(my_dataset) # create your dataloader

In [13]:
v_x = torch.stack([torch.Tensor(i) for i in val_x]) # transform to torch tensors
v_y = torch.stack([torch.Tensor(i) for i in val_y])
my_dataset = utils.TensorDataset(v_x,v_y) # create your datset
val_dataloader = utils.DataLoader(my_dataset) # create your dataloader

In [14]:
te_x = torch.stack([torch.Tensor(i) for i in test_x]) # transform to torch tensors
te_y = torch.stack([torch.Tensor(i) for i in test_y])
my_dataset = utils.TensorDataset(te_x,te_y) # create your datset
test_dataloader = utils.DataLoader(my_dataset) # create your dataloader

In [15]:
t_x.shape, t_y.shape, v_x.shape, v_y.shape, te_x.shape, te_y.shape

(torch.Size([1500, 2048]),
 torch.Size([1500, 1]),
 torch.Size([300, 2048]),
 torch.Size([300, 1]),
 torch.Size([195, 2048]),
 torch.Size([195, 1]))

In [18]:
for epoch in range(1, 100): ## run the model for 10 epochs
    train_loss, valid_loss = [], []
    ## training part 
    model.train()
    for data, target in train_dataloader:
        optimizer.zero_grad()
        ## 1. forward propagation
        output = model(data)
        
        ## 2. loss calculation
        loss = criterion(output, target)
        
        ## 3. backward propagation
        loss.backward()
        
        ## 4. weight optimization
        optimizer.step()
        
        train_loss.append(loss.item())
        
    ## evaluation part 
    model.eval()
    for data, target in val_dataloader:
        output = model(data)
        loss = criterion(output, target)
        valid_loss.append(loss.item())
    print ("Epoch:", epoch, "Training Loss: ", np.mean(train_loss), "Valid Loss: ", np.mean(valid_loss))


C:\Users\praty\Anaconda3\lib\site-packages\torch\nn\functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 1 Training Loss:  1.1462979247266816 Valid Loss:  0.658735742966334
Epoch: 2 Training Loss:  0.6445913439691067 Valid Loss:  0.6447298125425974
Epoch: 3 Training Loss:  0.631910047069192 Valid Loss:  0.6414164567987124
Epoch: 4 Training Loss:  0.6233120981454849 Valid Loss:  0.6411884177724521
Epoch: 5 Training Loss:  0.6221676084597906 Valid Loss:  0.6417685222625732
Epoch: 6 Training Loss:  0.6210772732496261 Valid Loss:  0.6422578972578049
Epoch: 7 Training Loss:  0.6283951601063211 Valid Loss:  0.6423569172620773
Epoch: 8 Training Loss:  0.6209591676990192 Valid Loss:  0.6425964480638504
Epoch: 9 Training Loss:  0.620936609407266 Valid Loss:  0.6427202677726745
Epoch: 10 Training Loss:  0.6268601865358651 Valid Loss:  0.642654475569725
Epoch: 11 Training Loss:  0.6274855069428061 Valid Loss:  0.6424791398644447
Epoch: 12 Training Loss:  0.6211905394196511 Valid Loss:  0.6426678097248077
Epoch: 13 Training Loss:  0.6227892467578252 Valid Loss:  0.6427657973766326
Epoch: 14 Tr

KeyboardInterrupt: 

In [19]:
torch.save(model, './resnet_transfer.pt')

C:\Users\praty\Anaconda3\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [16]:
for epoch in range(1, 100): ## run the model for 10 epochs
    train_loss, valid_loss = [], []
    ## training part 
    model.train()
    for data, target in train_dataloader:
        optimizer.zero_grad()
        ## 1. forward propagation
        output = model(data)
        
        ## 2. loss calculation
        loss = criterion(output, target)
        
        ## 3. backward propagation
        loss.backward()
        
        ## 4. weight optimization
        optimizer.step()
        
        train_loss.append(loss.item())
        
    ## evaluation part 
    model.eval()
    for data, target in val_dataloader:
        output = model(data)
        loss = criterion(output, target)
        valid_loss.append(loss.item())
    print ("Epoch:", epoch, "Training Loss: ", np.mean(train_loss), "Valid Loss: ", np.mean(valid_loss))


C:\Users\praty\Anaconda3\lib\site-packages\torch\nn\functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 1 Training Loss:  0.6592349473051727 Valid Loss:  0.6193452485402425
Epoch: 2 Training Loss:  0.6444176991457741 Valid Loss:  0.6194870723287265
Epoch: 3 Training Loss:  0.6427005646427473 Valid Loss:  0.6226571521162987
Epoch: 4 Training Loss:  0.6497400937958931 Valid Loss:  0.6277181524038314
Epoch: 5 Training Loss:  0.6336629179219405 Valid Loss:  0.6203315488497416
Epoch: 6 Training Loss:  0.6298922127783299 Valid Loss:  0.6169438596566518
Epoch: 7 Training Loss:  0.6285235941310724 Valid Loss:  0.6155502512057622
Epoch: 8 Training Loss:  0.6281950875024 Valid Loss:  0.6150892426570257
Epoch: 9 Training Loss:  0.6262563546895981 Valid Loss:  0.6147003951668739
Epoch: 10 Training Loss:  0.635164418688044 Valid Loss:  0.6144947161277136
Epoch: 11 Training Loss:  0.6263223273952802 Valid Loss:  0.6145180338621139
Epoch: 12 Training Loss:  0.6280725013166666 Valid Loss:  0.6144808762272199
Epoch: 13 Training Loss:  0.6310700549309453 Valid Loss:  0.6146284881234169
Epoch: 14 Tr